# Imports

In [1]:
import numpy as np
from scipy.optimize import minimize
import random
import multiprocessing
import itertools

import cvxpy as cp

In [2]:
np.random.seed(1)

# Create random Reachable sets

In [22]:
c = 5 # cache capacity
k = 5 # each user should be in at most K reachable sets
m = 2 # at most M items can be initially recommended to each user
L = 5 # the total number of items that the user will receive either through 
# direct recommendation or indirectly (because it is in reachable sets of users to whom items 
#have been recommended) should not be more than L
number_of_items = 10
number_of_users = 20
items= np.array(range(0,number_of_items)) # videos
users = np.array(range(0,number_of_users)) # users
np.random.shuffle(items)
np.random.shuffle(users)
# initialize x0 for the optimization
x = np.full(len(items),0.5)
y = np.full((len(items), len(users)),0.5)
x0 = np.concatenate([x.flatten(), y.flatten()])
# Random Reachable sets
reachable_sets = np.array(np.array([[np.array(random.sample(list(users), random.choice(users))) for user in users] for i in items]))
reachable_sets_dict = {}
for i in items:
    dic = {}
    for user in users:
        dic[user] = np.array(random.sample(list(users), random.choice(users)))
    reachable_sets_dict[i] = dic
        

I = np.zeros((len(items), len(users), len(users)))
for i in items:
    for u in users:
        for v in reachable_sets[i][u]:
            I[i][u][v] = 1

# Save the size of each intersection of reachable sets to a numpy array

In [23]:
intersections = np.zeros((len(items),len(users),len(users)))
for i in items:
    for u in users:
        for v in users:
            if v!=u:
                intersections[i][u][v] = len(set(reachable_sets[i][u])&set(reachable_sets[i][v]))

# Save the size of each reachable set to a numpy array

In [24]:
size_of_sets = np.vectorize(len)
reachable_sets_len = size_of_sets(reachable_sets)

# Save the size of each intersection of reachable sets to a dictionary

In [6]:
# intersections = np.zeros((len(items),len(users),len(users)))
# intersections = {}
# for item,row in reachable_sets_dict.items():
#     dic1={}
#     for user1,x in row.items():
#         dic2={}
#         for user2,y in row.items():
#             dic2[user2] = len(set(x)&set(y))
#         dic1[user1]=dic2
#     intersections[item]=dic1
            
        

In [70]:
reachable_sets_len.shape

(10, 12)

In [9]:
x.shape

(10,)

In [11]:
x[np.newaxis].T.shape

(1, 10)

In [73]:
((reachable_sets_len*y).sum(axis=0))[np.newaxis].shape

(1, 12)

In [74]:
np.dot(x[np.newaxis].T,((reachable_sets_len*y).sum(axis=0))[np.newaxis]).sum(axis=1).shape

(10,)

In [15]:
reachable_sets_len.shape

(10, 12)

In [25]:
y.shape

(10, 12)

In [14]:
intersections.shape

(10, 12, 12)

In [23]:
np.dot(y,intersections.T).shape

(10, 12, 10)

In [36]:
np.einsum('ijk,ik->ij',intersections,y).shape

(10, 12)

In [38]:
np.dot(y,np.einsum('ijk,ik->ij',intersections,y).T).shape

(10, 10)

In [39]:
reachable_sets_len.shape

(10, 12)

In [48]:
reachable_sets_len

array([[ 6,  8,  7,  3,  5, 10,  5,  5,  8,  8, 10,  6],
       [10,  7,  0,  1, 10, 10,  7,  4,  0,  7, 10, 11],
       [ 3,  3,  7,  7,  9, 11,  8, 10,  8,  7,  4,  0],
       [ 7,  6,  2,  0,  8, 10,  1, 10,  2,  3,  7,  5],
       [ 0, 11,  5,  5,  8,  9,  4,  0,  1, 10,  8,  3],
       [ 7, 11,  1,  8,  9,  9, 10,  9,  8,  9,  0,  9],
       [ 2,  5,  2,  9,  7,  7,  5, 11,  4, 10, 11,  4],
       [ 4,  2,  2,  4,  8,  1,  8,  8,  4,  4,  3,  3],
       [ 4,  2, 11,  4,  2, 11,  2,  9,  7,  7, 11,  9],
       [11,  5,  9,  9,  0,  1,  5,  1,  5,  1,  9,  2]])

In [44]:
y

array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]])

# Objective function

In [17]:
def objective(z):
#     x = z[:len(items)][np.newaxis]
#     y = z[len(items):].reshape(len(items),len(users))[np.newaxis]
    
    x = z[:len(items)]
    y = z[len(items):].reshape(len(items),len(users))
    #print("KAPPA")
    
    #The first term in (1) counts the users to which an item is 
    #initially recommended and the users in the ensuing reachable sets
    #first_term = sum([(1+len(reachable_sets[i][u])) * y[i][u] for u in users])
    #The second term in (1) is due to the overlap of reachable sets Riu,Riv
    #second_term = 0.5*sum([sum([len(set(reachable_sets[i][u])&set(reachable_sets[i][v]))*y[i][u]*y[i][v] for v in users if v!=u]) for u in users])
    # Ekfonisi
    #return -sum([x[i]*(sum([(1+len(reachable_sets[i][u])) * y[i][u] for u in users]) - 0.5*sum([sum([len(set(reachable_sets[i][u])&set(reachable_sets[i][v]))*y[i][u]*y[i][v] for v in users if v!=u]) for u in users])) for i in items])
    # Paper
    #return x.sum()
    #return -(np.dot(x,((((1+reachable_sets_len)*y)-0.5*()).sum(axis=0)))).sum(axis=1)
    #return -sum([x[i]*sum([(1+len(reachable_sets[i][u])) * y[i][u] - 0.5*sum([len(set(reachable_sets[i][u])&set(reachable_sets[i][v]))*y[i][u]*y[i][v] for v in users if v!=u]) for u in users]) for i in items])
    #return -sum([x[i]*sum([(1+reachable_sets_len[i][u]) * y[i][u] - 0.5*sum([intersections[i][u][v]*y[i][u]*y[i][v] for v in users if v!=u]) for u in users]) for i in items])
    
    
    #return -(np.dot(x,(np.dot((1+reachable_sets_len),y.T)-0.5*(np.dot(y,np.einsum('ijk,ik->ij',intersections,y).T)))))
    return -np.dot(x,(((1+reachable_sets_len)*y).sum(axis=1) - 0.5*(y*((y[:,:,None]*intersections).sum(axis=2))).sum(axis=1)))

In [70]:
np.dot(x,np.dot((1+reachable_sets_len),y.T)).shape

(10,)

In [98]:
np.dot(x,(((1+reachable_sets_len)*y).sum(axis=1) - 0.5*(y*((y[:,:,None]*intersections).sum(axis=2))).sum(axis=1)))

-36.25

In [93]:
(y*((y[:,:,None]*intersections).sum(axis=2))).sum(axis=1).shape

(10,)

# Constraint (2) is the cache capacity constraint

In [7]:
def constraint2(z):  
    x = z[:len(items)]
    
    return sum([x[i] for i in items]) - c

# Constraint (3) says that at most M items can be initially recommended to each user

In [8]:
def constraint3(z):  
    y = z[len(items):].reshape(len(items),len(users))
    
    return [(-sum([y[i][u] for i in items]))+m for u in users]

# Constraint (4) says that each user should be in at most K reachable sets these capture content consumption, attention span or time constraints

In [9]:
def constraint4(z):  
    y = z[len(items):].reshape(len(items),len(users))
    
    return [-sum([sum([I[i][u][v]*y[i][u] for u in users]) for i in items])+k for u in users]

# Constraint (6) says that the total number of items that the user will receive either through direct recommendation or indirectly (because it is in reachable sets of users to whom items have been recommended) should not be more than L

In [18]:
def constraint6(z):
    y = z[len(items):].reshape(len(items),len(users))
    

    return [-sum([y[i][u] for i in items]) - sum([sum([I[i][u][v]*y[i][u] for v in users]) for i in items])+L for u in users]

# Solve

In [25]:
con2 = {'type': 'eq', 'fun': constraint2} 
con3 = {'type': 'ineq', 'fun': constraint3} 
con4 = {'type': 'ineq', 'fun': constraint4} 
con6 = {'type': 'ineq', 'fun': constraint6}
cons = ([con2, con6])
# Bounds
b = (0.0,1.0)
bnds = [b for i in range(len(x0))]

# show initial objective
print('Initial Objective: ' + str(-1*objective(x0)))

solution = minimize(objective, x0, method='SLSQP', bounds=bnds, constraints=cons)

Initial Objective: -663.375


In [26]:
x = solution.x

# show final objective
print('Final Objective: ' + str(-1*objective(x)))

Final Objective: 63.70074829910546


In [27]:
solution

     fun: -63.70074829910546
     jac: array([-1.48198175e+00, -1.17928534e+01, -3.17811966e-03, -1.39300027e+01,
       -3.21531296e-03, -1.62458420e-03, -1.35375791e+01, -1.82454586e-01,
       -1.24910808e+01, -1.19492326e+01,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  3.93867493e-04, -6.93545723e+00,
       -2.23636627e-04, -5.04682541e+00, -8.52493286e-01, -7.54303980e+00,
       -1.91211700e-04, -6.19055271e-01, -4.42028046e-04, -2.01327944e+00,
       -4.37641525e+00, -7.19286251e+00, -1.81327295e+00, -7.31444693e+00,
       -1.11677170e-01, -5.84332323e+00, -6.88600969e+00, -4.66939497e+00,
       -6.47834015e+00, -1.34791422e+00,  0.00000000e+00,  0.

In [107]:
solution.x.shape

(130,)

In [108]:
x = solution.x[:len(items)].round()
y = solution.x[len(items):].reshape(len(items),len(users)).round()

# Solution

## Items in cache

In [109]:
x

array([0., 1., 0., 1., 1., 0., 0., 1., 0., 1.])

In [110]:
x.sum()

5.0

In [111]:
x.shape

(10,)

## Recommended items

In [112]:
y

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.]])

In [113]:
y.sum(axis=0) # items recommended to each user

array([1., 0., 1., 1., 1., 0., 0., 2., 1., 0., 1., 1.])

In [114]:
y.shape

(10, 12)

In [27]:
# x = cp.Variable(x.shape)
# y = cp.Variable(y.shape)
# prob = cp.Problem(cp.Maximize(cp.sum(x*cp.sum())),
#                  [x<=1,
#                   x>=0,
#                   y<=1,
#                   y>=0
                     
# #                   sum([x[i] for i in items]) == c,
# #                  [sum([y[i][u] for i in items]) <= m for u in users],
# #                  [sum([sum([I[i][u][v]*y[i][u] for u in users]) for i in items]) <= k for u in users]
#                   ])
# prob.solve()